In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.cross_validation import cross_val_score

C:\Users\shekh\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
dataFrame = pd.read_csv('C:/Users/shekh/Desktop/rentallisting/extrapolated_trained_data_new_1.csv');
df = pd.DataFrame(dataFrame);

In [3]:
df['interest_level'] = df['interest_level'].astype('category')
df['interest_level_codes'] = df['interest_level'].cat.codes

df['display_address'] = df['display_address'].astype('category')
df['display_address_codes'] = df['display_address'].cat.codes

df['latitude'] = df['latitude'].astype('category')
df['latitude_codes'] = df['latitude'].cat.codes

df['longitude'] = df['longitude'].astype('category')
df['longitude_codes'] = df['longitude'].cat.codes

In [4]:
featuresDF = df.drop('interest_level', axis=1)
featuresDF = featuresDF.drop('building_id', axis=1)
featuresDF = featuresDF.drop('created', axis=1)
featuresDF = featuresDF.drop('description', axis=1)
featuresDF = featuresDF.drop('display_address', axis=1)
featuresDF = featuresDF.drop('latitude', axis=1)
featuresDF = featuresDF.drop('longitude', axis=1)
featuresDF = featuresDF.drop('manager_id', axis=1)
featuresDF = featuresDF.drop('photos', axis=1)
featuresDF = featuresDF.drop('street_address', axis=1)
featuresDF = featuresDF.drop('listing_id', axis=1)

featuresDF.shape

(45342, 289)

In [5]:
#2. Using Variance Threshold for feature selection
selector = VarianceThreshold(threshold=(.9 * (1 - .9)));
temp = selector.fit_transform(featuresDF);

In [6]:
type(temp)

numpy.ndarray

In [7]:
temp.shape

(45342L, 22L)

In [8]:
selected_features = selector.get_support(indices=True)

In [9]:
reducedFeatureDF = featuresDF.iloc[:,selected_features]

In [10]:
reducedFeatureDF.columns

Index([u'bathrooms', u'bedrooms', u'no_of_photos', u'price', u'dish_washer',
       u'fitness_center', u'hardwood_floors', u'dining_room', u'elevator',
       u'doorman', u'dog_allowed', u'roof_deck', u'laundry', u'pre_war',
       u'cat_allowed', u'outdoor_space', u'laundry_bldg', u'no_fee',
       u'interest_level_codes', u'display_address_codes', u'latitude_codes',
       u'longitude_codes'],
      dtype='object')

In [11]:
y_label = reducedFeatureDF['interest_level_codes']

In [12]:
# X_train, X_test, y_train, y_test = train_test_split(reducedFeatureDF,y_label, test_size=0.40,random_state=15,stratify=y_label)
train, validate, test = np.split(reducedFeatureDF.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
print len(train);
print len(validate);
print len(test);
X_train = train.drop('interest_level_codes', axis=1)
X_validate = validate.drop('interest_level_codes', axis=1)
X_test = test.drop('interest_level_codes', axis=1)

y_train = train['interest_level_codes']
y_validate = validate['interest_level_codes']
y_test = test['interest_level_codes']

print len(X_train)
print len(X_validate)
print len(X_test)
print len(y_train)
print len(y_validate)
print len(y_test)

27205
9068
9069
27205
9068
9069
27205
9068
9069


In [13]:
print y_train.value_counts();
print y_validate.value_counts();
print y_test.value_counts();
target_names = ['0', '1', '2'];

1    19363
2     5951
0     1891
Name: interest_level_codes, dtype: int64
1    6420
2    2016
0     632
Name: interest_level_codes, dtype: int64
1    6433
2    1985
0     651
Name: interest_level_codes, dtype: int64


In [14]:
svmLinearKernelClassifier = svm.LinearSVC();
svmLinearKernelClassifier.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [15]:
predicted_labels_validation = svmLinearKernelClassifier.predict(X_validate)

In [16]:
print(classification_report(y_validate, predicted_labels_validation, target_names=target_names))

             precision    recall  f1-score   support

          0       1.00      0.00      0.00       632
          1       0.71      1.00      0.83      6420
          2       0.00      0.00      0.00      2016

avg / total       0.57      0.71      0.59      9068



In [17]:
predicted_labels_test = svmLinearKernelClassifier.predict(X_test)

In [18]:
print(classification_report(y_test, predicted_labels_test, target_names=target_names))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00       651
          1       0.71      1.00      0.83      6433
          2       0.00      0.00      0.00      1985

avg / total       0.50      0.71      0.59      9069



C:\Users\shekh\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
